In [2]:
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord

new_record = []
for seq_record in SeqIO.parse('/home/ouconstand/data/Virus_Host/seq_data/sequence_0.fasta','fasta'):
    seq = seq_record.seq
    if len(seq) > 2000:
        for i in range(0, len(seq), 500):
            if i + 2000 > len(seq):
                new_seq = seq[-2000:]
                rec = SeqRecord(new_seq, id=seq_record.id, description=seq_record.description, name=seq_record.name)
                new_record.append(rec)
                break

            new_seq = seq[i:i+2000]
            rec = SeqRecord(new_seq,id=seq_record.id,description=seq_record.description,name=seq_record.name)
            new_record.append(rec)

    else:
        print('sequence length is less than 2000 ' + seq_record.name)

sequence length is less than 2000 MT221943.1
sequence length is less than 2000 MG000871.1
sequence length is less than 2000 KX286039.1
sequence length is less than 2000 MT222095.1
sequence length is less than 2000 KJ690965.1
sequence length is less than 2000 MG591491.1
sequence length is less than 2000 MT221904.1
sequence length is less than 2000 KX425844.1
sequence length is less than 2000 MT221761.1
sequence length is less than 2000 MT221744.1
sequence length is less than 2000 MT221762.1
sequence length is less than 2000 MF769465.1
sequence length is less than 2000 MT221870.1
sequence length is less than 2000 MF769474.1
sequence length is less than 2000 MG963196.1
sequence length is less than 2000 MG762664.1


In [3]:
new_record[0]

SeqRecord(seq=Seq('ACTTTCGATCTCTTGTAGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGC...GCT'), id='MZ519413.1', name='MZ519413.1', description='MZ519413.1 Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/USA/GA-CDC-GA-EHC-081C/2020 ORF1ab polyprotein (ORF1ab), ORF1a polyprotein (ORF1ab), surface glycoprotein (S), ORF3a protein (ORF3a), envelope protein (E), membrane glycoprotein (M), ORF6 protein (ORF6), ORF7a protein (ORF7a), ORF7b (ORF7b), ORF8 protein (ORF8), nucleocapsid phosphoprotein (N), and ORF10 protein (ORF10) genes, complete cds', dbxrefs=[])

In [6]:
print(new_record[0].seq)
print(new_record[0].id)

ACTTTCGATCTCTTGTAGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTCGGCTGCATGCTTAGTGCACTCACGCAGTATAATTAATAACTAATTACTGTCGTTGACAGGACACGAGTAACTCGTCTATCTTCTGCAGGCTGCTTACGGTTTCGTCCGTGTTGCAGCCGATCATCAGCACATCTAGGTTTTGTCCGGGTGTGACCGAAAGGTAAGATGGAGAGCCTTGTCCCTGGTTTCAACGAGAAAACACACGTCCAACTCAGTTTGCCTGTTTTACAGGTTCGCGACGTGCTCGTACGTGGCTTTGGAGACTCCGTGGAGGAGGTCTTATCAGAGGCACGTCAACATCTTAAAGATGGCACTTGTGGCTTAGTAGAAGTTGAAAAAGGCGTTTTGCCTCAACTTGAACAGCCCTATGTGTTCATCAAACGTTCGGATGCTCGAACTGCACCTCATGGTCATGTTATGGTTGAGCTGGTAGCAGAACTCGAAGGCATTCAGTACGGTCGTAGTGGTGAGACACTTGGTGTCCTTGTCCCTCATGTGGGCGAAATACCAGTGGCTTACCGCAAGGTTCTTCTTCGTAAGAACGGTAATAAAGGAGCTGGTGGCCATAGTTACGGCGCCGATCTAAAGTCATTTGACTTAGGCGACGAGCTTGGCACTGATCCTTATGAAGATTTTCAAGAAAACTGGAACACTAAACATAGCAGTGGTGTTACCCGTGAACTCATGCGTGAGCTTAACGGAGGGGCATACACTCGCTATGTCGATAACAACTTCTGTGGCCCTGATGGCTACCCTCTTGAGTGCATTAAAGACCTTCTAGCACGTGCTGGTAAAGCTTCATGCACTTTGTCCGAACAACTGGACTTTATTGACACTAAGAGGGGTGTATACTGCTGCCGTGAACATGAGCATGAAATTGCTTGGTACACGGAACGTTCTGAAAAGAGCTATGAATTGCAGACACCTTTTGAAATTAA

In [15]:
nucl_dict = dict([(value, key) for key, value in enumerate(['A', 'T', 'C', 'G'])])

In [25]:
import numpy as np

def nucl_to_int(sequences, dim=4):
    results = np.zeros((len(sequences),dim))
    sequences = [nucl_dict.get(seq) for seq in sequences]
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1
    return results

In [42]:
train_data = np.asarray([nucl_to_int(seq.seq) for seq in new_record])

In [43]:
train_data.shape

(223, 2000, 4)

In [48]:
train_data = train_data.reshape(223, 2000*4)

In [33]:
from keras.utils.np_utils import to_categorical

labels_dict = dict([(value, key) for key, value in enumerate(['KT368856.1', 'KT368895.1', 'MT726043.1', 'MZ519413.1'])])
labels_list = [labels_dict.get(seq.id) for seq in new_record]
train_labels = to_categorical(labels_list)

In [39]:
train_labels.shape

(223, 4)

In [49]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(8000,)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(4, activation='softmax'))

In [50]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [52]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 64)                512064    
_________________________________________________________________
dense_4 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_5 (Dense)              (None, 4)                 260       
Total params: 516,484
Trainable params: 516,484
Non-trainable params: 0
_________________________________________________________________


In [55]:
model.fit(train_data, train_labels, epochs = 3)

Epoch 1/3
7/7 [==============================] - 0s 52ms/step - loss: 1.2494 - accuracy: 0.3901
Epoch 2/3
7/7 [==============================] - 0s 49ms/step - loss: 1.0627 - accuracy: 0.5067
Epoch 3/3
7/7 [==============================] - 0s 41ms/step - loss: 0.6697 - accuracy: 0.7803
